### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","Production")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read.json(
    f"{bronze_folder_path}/{v_file_date}/country.json",
    schema=countries_schema)

### Paso 2 - Eliminar las columnas no deseadas del Dataframe

In [0]:
countries_dropped_df = countries_df.drop("countryIsoCode")

Paso 3 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
countries_final_df = countries_dropped_df\
    .withColumnsRenamed({"countryId":"country_id", "countryName":"country_name"})\
    .withColumns({"environment": lit("Production"), "file_date": lit(v_file_date)})                   

In [0]:
countries_final_df = add_ingestion_date(countries_final_df)

### Paso 4 - Escribir la salida en formato Parquet

In [0]:
countries_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.country")

In [0]:
%sql
select * from movie_silver.country